In [1]:
import pandas as pd
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer

log_path = 'data/purchasing_example.xes'
log_alpha_path = 'results/bisection/log_alpha.csv'

log = xes_importer.apply(log_path)
log_df = pm4py.convert_to_dataframe(log)

log_alpha = pd.read_csv(log_alpha_path)
# log = pm4py.filter_event_attribute_values(log, 'lifecycle:transition', ['complete'], level="event", retain=True)

C:\Users\vinci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 608/608 [00:00<00:00, 664.39it/s]


In [2]:
log_alpha

,Resource,Activity,org:resource,concept:name,lifecycle:transition,time:timestamp,start:timestamp,case:creator,case:variant-index,case:variant,case:concept:name
0,Kim Passa,Create Purchase Requisition,Kim Passa,Create Purchase Requisition,complete,2011-01-01 06:37:00+01:00,2011-01-01 06:37:00+01:00,Fluxicon Disco,2,Variant 2,1
1,Kim Passa,Create Request for Quotation,Kim Passa,Create Request for Quotation,complete,2011-01-01 11:45:00+01:00,2011-01-01 09:42:14.062500+01:00,Fluxicon Disco,2,Variant 2,1
2,Karel de Groot,Analyze Request for Quotation,Karel de Groot,Analyze Request for Quotation,complete,2011-01-01 12:55:00+01:00,2011-01-01 12:55:00+01:00,Fluxicon Disco,2,Variant 2,1
3,Karel de Groot,Send Request for Quotation to Supplier,Karel de Groot,Send Request for Quotation to Supplier,complete,2011-01-01 18:09:00+01:00,2011-01-01 17:53:28.784180+01:00,Fluxicon Disco,2,Variant 2,1
4,Magdalena Predutta,Create Quotation comparison Map,Magdalena Predutta,Create Quotation comparison Map,complete,2011-01-01 22:03:00+01:00,2011-01-01 21:38:05.507812+01:00,Fluxicon Disco,2,Variant 2,1
...,...,...,...,...,...,...,...,...,...,...,...
9114,Heinz Gutschmidt,Analyze Purchase Requisition,Heinz Gutschmidt,Analyze Purchase Requisition,complete,2011-10-13 02:02:00+02:00,2011-10-13 02:02:00+02:00,Fluxicon Disco,4,Variant 4,1941
9115,Francis Odell,Create Request for Quotation,Francis Odell,Create Request for Quotation,complete,2011-10-13 02:09:00+02:00,2011-10-13 02:09:00+02:00,Fluxicon Disco,4,Variant 4,1941
9116,Karel de Groot,Analyze Request for Quotation,Karel de Groot,Analyze Request for Quotation,complete,2011-10-14 21:18:00+02:00,2011-10-14 21:18:00+02:00,Fluxicon Disco,4,Variant 4,1941
9117,Tesca Lobes,Create Purchase Requisition,Tesca Lobes,Create Purchase Requisition,complete,2011-10-13 14:21:00+02:00,2011-10-13 14:21:00+02:00,Fluxicon Disco,3,Variant 3,1949


In [3]:
log_df.sort_values(by='time:timestamp', inplace=True)
log_df = log_df[log_df['lifecycle:transition'] != 'complete']
log_df.index = range(len(log_df))

log_alpha.sort_values(by='time:timestamp', inplace = True)
log_alpha.index = range(len(log_alpha))

In [4]:
import re

log_alpha['start:timestamp'] = log_alpha['start:timestamp'].apply(lambda x: re.sub('\.[0-9]+', '', str(x)))

In [9]:
(log_df['time:timestamp'] - pd.to_datetime(log_alpha['start:timestamp'])).apply(lambda x: x.total_seconds()).abs().mean()

C:\Users\vinci\AppData\Local\Temp\ipykernel_6036\1899388007.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  (log_df['time:timestamp'] - pd.to_datetime(log_alpha['start:timestamp'])).apply(lambda x: x.total_seconds()).abs().mean()


11126.747011733743

In [10]:
11126/60

185.43333333333334

In [11]:
185/60

3.0833333333333335

In [12]:
log_path = 'data/purchasing_example.xes'
log_alpha_path = 'results/single/log_alpha_one.csv'

log = xes_importer.apply(log_path)
log_df = pm4py.convert_to_dataframe(log)

log_alpha = pd.read_csv(log_alpha_path)
# log = pm4py.filter_event_attribute_values(log, 'lifecycle:transition', ['complete'], level="event", retain=True)

log_df.sort_values(by='time:timestamp', inplace=True)
log_df = log_df[log_df['lifecycle:transition'] != 'complete']
log_df.index = range(len(log_df))

log_alpha.sort_values(by='time:timestamp', inplace = True)
log_alpha.index = range(len(log_alpha))

import re

log_alpha['start:timestamp'] = log_alpha['start:timestamp'].apply(lambda x: re.sub('\.[0-9]+', '', str(x)))

parsing log, completed traces :: 100%|██████████| 608/608 [00:00<00:00, 721.22it/s]


In [13]:
(log_df['time:timestamp'] - pd.to_datetime(log_alpha['start:timestamp'])).apply(lambda x: x.total_seconds()).abs().mean()

C:\Users\vinci\AppData\Local\Temp\ipykernel_6036\1899388007.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  (log_df['time:timestamp'] - pd.to_datetime(log_alpha['start:timestamp'])).apply(lambda x: x.total_seconds()).abs().mean()


13254.907774975327

In [15]:
13254.907774975327/60/60

3.6819188263820353

In [16]:
0.6*60

36.0

In [20]:
log_path = 'data/purchasing_example.xes'
log_alpha_path = 'results/alpha_1/log_alpha.csv'

log = xes_importer.apply(log_path)
log_df = pm4py.convert_to_dataframe(log)

log_alpha = pd.read_csv(log_alpha_path)
# log = pm4py.filter_event_attribute_values(log, 'lifecycle:transition', ['complete'], level="event", retain=True)

log_df.sort_values(by='time:timestamp', inplace=True)
log_df = log_df[log_df['lifecycle:transition'] != 'complete']
log_df.index = range(len(log_df))

log_alpha.sort_values(by='time:timestamp', inplace = True)
log_alpha.index = range(len(log_alpha))

import re

log_alpha['start:timestamp'] = log_alpha['start:timestamp'].apply(lambda x: re.sub('\.[0-9]+', '', str(x)))

parsing log, completed traces :: 100%|██████████| 608/608 [00:00<00:00, 859.00it/s]


In [21]:
(log_df['time:timestamp'] - pd.to_datetime(log_alpha['start:timestamp'])).apply(lambda x: x.total_seconds()).abs().mean()

C:\Users\vinci\AppData\Local\Temp\ipykernel_6036\1899388007.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  (log_df['time:timestamp'] - pd.to_datetime(log_alpha['start:timestamp'])).apply(lambda x: x.total_seconds()).abs().mean()


36978.001973900646

In [22]:
36978/60/60

10.271666666666667